# Custom Video Object Detection

In [1]:
""" import dependencies / necessary packages """
import scipy
import time 
import math 
import pandas as pd
import numpy as np
import wave, struct
import matplotlib.pyplot as plt
# import the necessary packages
from collections import deque
import numpy as np
import argparse
import cv2

from IPython.display import display
from scipy.io import wavfile as wav
from scipy import signal

# Show matplotlib plots inline (nicely formatted in the notebook)
%matplotlib inline

print ('done')

done


## To-Do
    1. Get image snips
        - Ears, Nose, Tail
    2. Download and train classifier
    3. Write code to do live classifying 
    ____________________________________
    
    4. Edit code to get coordinates 
    5. Use coordinates to get directions, relative positions
        - facing, proximity relative to body size, alignment

## Get Video images from video to label

In [10]:
# https://stackoverflow.com/questions/33650974/opencv-python-read-specific-frame-using-videocapture
import numpy as np
import cv2

#Get video name from user
#Ginen video name must be in quotes, e.g. "pirkagia.avi" or "plaque.avi"
#Please give the video name including its extension. E.g. \"pirkagia.avi\":\n
# video_name = input("Please give the video name including its extension. E.g. \"pirkagia.avi\":\n")

video_name = (r"C:/Users/leviz/OneDrive/School/PhD/projects/Video/260104_100607_24hREUNION.avi")

""" set desired image type (jpg, png)"""
image_type = "jpg"
#Open the video file
cap = cv2.VideoCapture(video_name)


if not cap.isOpened(): 
    print ("could not open video_name")
    

#Entire video length information
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)

print ("Number of Frame:")
print (num_frames)
print ("Image Width:")
print (width)
print ("Image Height:")
print (height)
print ("Frames-Per-Second:")
print (fps)

#Set frame_no in range 0.0-1.0
#For example, if we have a video of 30 seconds having 25 frames per seconds, we thus would have 750 frames.
#The examined frame must get a value from 0 to 749.
#For more info about the video flags see here: https://stackoverflow.com/questions/11420748/setting-camera-parameters-in-opencv-python
#Here we calculate the last frame as frame sequence=749. 

#In case you want to select other frame change value 749.
#BE CAREFUL! Each video has different time length and frame rate. 
#So make sure that you have the right parameters for the right video!

#Comment out and change to desired length if the orignial is not what you want to sample
#Example: change fps to 10 if original is 20fps and you want to sample at 10fps
# time_length = 30.0
# fps=.00001
# frame_seq = time_length*fps-1
# frame_no = (frame_seq /(time_length*fps))

time_length = num_frames / fps
frame_seq = num_frames-1
frame_no = (frame_seq /(time_length*fps))

Number of Frame:
40223
Image Width:
640
Image Height:
480
Frames-Per-Second:
30.0


In [22]:
count=1
while(cap.isOpened()):
    frameId= cap.get(1)
    #Read the next frame from the video. If you set frame 749 above then the code will return the last frame.
    ret, frame = cap.read()
    if ret == True:
        if (count % math.floor(fps*2) != 0):
            count+=1
            
        if (count % math.floor(fps*2) == 0):
            # define range of blue, red, green color in HSV
    #         lower_red = np.array([20,0,0])
    #         upper_red = np.array([255,255,255])

            lower_green = np.array([0,0,0])
            upper_green = np.array([255,255,255])

            lower_blue = np.array([0,0,0])
            upper_blue = np.array([255,255,170])

            #Set grayscale or HSV colorspace for the frame. 
    #         gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #         color = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            hsv4 = cv2.cvtColor(frame, cv2.COLOR_RGB2HLS)
    #         hsv7 = cv2.cvtColor(frame, cv2.COLOR_RGB2XYZ)

            # add color masks
    #         hsv_red = cv2.inRange(hsv, lower_red, upper_red)
    #         hsv_green = cv2.inRange(hsv, lower_green, upper_green)
            hsv_blue = cv2.inRange(hsv, lower_blue, upper_blue)

    #         res_red = cv2.bitwise_and(hsv4,hsv4, mask= hsv_red)
    #         res_green = cv2.bitwise_and(hsv4,hsv4, mask= hsv_green)
    #         res_blue = cv2.bitwise_and(hsv4,hsv4, mask= hsv_blue)

    #         res_red = cv2.bitwise_and(frame,frame, mask= hsv_red)
    #         res_green = cv2.bitwise_and(frame,frame, mask= hsv_green)
            res_blue = cv2.bitwise_and(frame,frame, mask= hsv_blue)        

            #Cut the video extension to have the name of the video
            my_video_name = video_name.split(".")[0]

    #         cv2.imshow("original", frame)
    #         cv2.imshow('HSV', hsv)
    #         cv2.imshow('HSV4', hsv4)
    # #         cv2.imshow('HSV7', hsv7)

    #         cv2.imshow("res red", res_red)
    #         cv2.imshow("res green", res_green)
    #         cv2.imshow("res blue", res_blue)

    #         cv2.imshow("red", hsv_red)
    #         cv2.imshow('green', hsv_green)
    #         cv2.imshow("blue", hsv_blue)
    #         cv2.imshow('green', hsv_green)

            my_video_name = video_name.split(".")[0]


            cv2.imwrite(my_video_name+ "_frame_"+str(count)+'.'+image_type,res_blue)

            count += 1
        if cv2.waitKey(5) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break
        
    else:
        break

## Video Writer

In [18]:

            
import numpy as np
import cv2

cap = cv2.VideoCapture('260104_100607_24hREUNION.avi')
fps = int(fps)
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter('output.mp4',fourcc, fps, (width,height))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        lower_blue = np.array([0,0,0])
        upper_blue = np.array([255,255,170])
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        hsv_blue = cv2.inRange(hsv, lower_blue, upper_blue)
        res_blue = cv2.bitwise_and(frame,frame, mask= hsv_blue)
        
        frame = res_blue
        # write the flipped frame
        out.write(frame)

#         cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()

Press [ESC] to quit demo


In [76]:
while(cap.isOpened()):

    #Read the next frame from the video. If you set frame 749 above then the code will return the last frame.
    ret, frame = cap.read()
    if ret == True:
        
        # define range of blue, red, green color in HSV
#         lower_red = np.array([20,0,0])
#         upper_red = np.array([255,255,255])
        
        lab= cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
        cv2.imshow("lab",lab)

        #-----Splitting the LAB image to different channels-------------------------
        l, a, b = cv2.split(lab)
        cv2.imshow('l_channel', l)
        cv2.imshow('a_channel', a)
        cv2.imshow('b_channel', b)

        #-----Applying CLAHE to L-channel-------------------------------------------
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        cl = clahe.apply(l)
        cv2.imshow('CLAHE output', cl)

        #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
        limg = cv2.merge((cl,a,b))
        cv2.imshow('limg', limg)
        
        #-----Converting image from LAB Color model to RGB model--------------------
        final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
        cv2.imshow('final', final)

        hsv4 = cv2.cvtColor(final, cv2.COLOR_RGB2HLS)

        lower_blue = np.array([0,0,0])
        upper_blue = np.array([255,255,120])
        hsv_blue = cv2.inRange(final, lower_blue, upper_blue)
        res_blue = cv2.bitwise_and(final,final, mask= hsv_blue)        
        cv2.imshow("res blue", res_blue)
        
        if cv2.waitKey(5) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break

https://docs.opencv.org/3.2.0/df/d9d/tutorial_py_colorspaces.html

In [ ]:


#The first argument of cap.set(), number 2, defines that parameter for setting the frame selection.
#Number 2 defines flag CV_CAP_PROP_POS_FRAMES which is a 0-based index of the frame to be decoded/captured next.
#The second argument defines the frame number in range 0.0-1.0
# cap.set(1,frame_no);
count=1
while(cap.isOpened()):

    #Read the next frame from the video. If you set frame 749 above then the code will return the last frame.
    ret, frame = cap.read()
    if ret == True:
        
        #Set grayscale or HSV colorspace for the frame. 
        
#         gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)


        
        #Cut the video extension to have the name of the video
        my_video_name = video_name.split(".")[0]

        #Uncomment the line below to Display the resulting frame
        # cv2.imshow(my_video_name+' frame '+ str(frame_seq),gray)
        
        #Store this frame to an image
#         cv2.imwrite(my_video_name+'_frame_'+str(frame_seq)+'.'+image_type,gray)
#         cv2.imwrite(my_video_name+'_frame_'+str(count)+'.'+image_type,gray)
        cv2.imwrite(my_video_name+ "_frame_"+str(count)+'.'+image_type,hsv)

        count += 1
#Set waitKey 
# cv2.waitKey()
        if cv2.waitKey(30) & 0xFF == ord('q'):
            print ("waitKey reached. Break.")
            break
    else:
        break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

# sign tracking - warped boxes

In [ ]:
""" sign tracking """

# import the necessary packages
import time
import cv2
import numpy as np

#Difference Variable
minDiff = 10000
minSquareArea = 5000
match = -1

#Frame width & Height
w=640
h=480

#Reference Images Display name & Original Name
#ReferenceImages = ["ArrowL.jpg","ArrowR.jpg","ArrowT.jpg","Ball.jpg","Go.jpg","Stop.jpg"]
#ReferenceTitles = ["Turn Left 90","Turn Right 90","Turn Around","Search for Ball","Start..","Stop!"]

#define class for References Images
class Symbol:
    def __init__(self):
        self.img = 0
        self.name = 0

#define class instances (6 objects for 6 different images)
symbol= [Symbol() for i in range(6)]



def readRefImages():
    for count in range(6):
        image = cv2.imread(ReferenceImages[count], cv2.COLOR_BGR2GRAY)
        symbol[count].img = cv2.resize(image,(w/2,h/2),interpolation = cv2.INTER_AREA)
        symbol[count].name = ReferenceTitles[count]
        #cv2.imshow(symbol[count].name,symbol[count].img);


def order_points(pts):
        # initialzie a list of coordinates that will be ordered
        # such that the first entry in the list is the top-left,
        # the second entry is the top-right, the third is the
        # bottom-right, and the fourth is the bottom-left
        rect = np.zeros((4, 2), dtype = "float32")

        # the top-left point will have the smallest sum, whereas
        # the bottom-right point will have the largest sum
        s = pts.sum(axis = 1)
        rect[0] = pts[np.argmin(s)]
        rect[2] = pts[np.argmax(s)]

        # now, compute the difference between the points, the
        # top-right point will have the smallest difference,
        # whereas the bottom-left will have the largest difference
        diff = np.diff(pts, axis = 1)
        rect[1] = pts[np.argmin(diff)]
        rect[3] = pts[np.argmax(diff)]

        # return the ordered coordinates
        return rect

def four_point_transform(image, pts):
        # obtain a consistent order of the points and unpack them
        # individually
        rect = order_points(pts)
        (tl, tr, br, bl) = rect

        maxWidth = w/2
        maxHeight = h/2

        dst = np.array([
                [0, 0],
                [maxWidth - 1, 0],
                [maxWidth - 1, maxHeight - 1],
                [0, maxHeight - 1]], dtype = "float32")

        # compute the perspective transform matrix and then apply it
        M = cv2.getPerspectiveTransform(rect, dst)
        warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

        # return the warped image
        return warped


def auto_canny(image, sigma=0.33):
        # compute the median of the single channel pixel intensities
        v = np.median(image)

        # apply automatic Canny edge detection using the computed median
        lower = int(max(0, (1.0 - sigma) * v))
        upper = int(min(255, (1.0 + sigma) * v))
        edged = cv2.Canny(image, lower, upper)

        # return the edged image
        return edged


def resize_and_threshold_warped(image):
        #Resize the corrected image to proper size & convert it to grayscale
        #warped_new =  cv2.resize(image,(w/2, h/2))
        warped_new_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        #Smoothing Out Image
        blur = cv2.GaussianBlur(warped_new_gray,(5,5),0)

        #Calculate the maximum pixel and minimum pixel value & compute threshold
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(blur)
        threshold = (min_val + max_val)/2

        #Threshold the image
        ret, warped_processed = cv2.threshold(warped_new_gray, threshold, 255, cv2.THRESH_BINARY)

        #return the thresholded image
        return warped_processed




def main():

    #Font Type
    font = cv2.FONT_HERSHEY_SIMPLEX


    # initialize the camera and grab a reference to the raw camera capture
    video = cv2.VideoCapture(0)

    #Windows to display frames
    cv2.namedWindow("Main Frame", cv2.WINDOW_AUTOSIZE)
    cv2.namedWindow("Matching Operation", cv2.WINDOW_AUTOSIZE)
    cv2.namedWindow("Corrected Perspective", cv2.WINDOW_AUTOSIZE)
    cv2.namedWindow("Contours", cv2.WINDOW_AUTOSIZE)

    #Read all the reference images
    readRefImages()

    # capture frames from the camera
    while True:
            ret, OriginalFrame = video.read()
            gray = cv2.cvtColor(OriginalFrame, cv2.COLOR_BGR2GRAY)
            blurred = cv2.GaussianBlur(gray,(3,3),0)

            #Detecting Edges
            edges = auto_canny(blurred)

            #Contour Detection & checking for squares based on the square area
            cntr_frame, contours, hierarchy = cv2.findContours(edges,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

            for cnt in contours:
                    approx = cv2.approxPolyDP(cnt,0.01*cv2.arcLength(cnt,True),True)

                    if len(approx)==4:
                            area = cv2.contourArea(approx)

                            if area > minSquareArea:
                                    cv2.drawContours(OriginalFrame,[approx],0,(0,0,255),2)
                                    warped = four_point_transform(OriginalFrame, approx.reshape(4, 2))
                                    warped_eq = resize_and_threshold_warped(warped)


                                    for i in range(6):
                                        diffImg = cv2.bitwise_xor(warped_eq, symbol[i].img)
                                        diff = cv2.countNonZero(diffImg);

                                        if diff < minDiff:
                                            match = i

                                            #print symbol[i].name, diff
                                            #print approx.reshape(4,2)[0]
                                            #cv2.putText(OriginalFrame,symbol[i].name, (10,30), font, 1, (255,0,255), 2, cv2.LINE_AA)
                                            cv2.putText(OriginalFrame,symbol[i].name, tuple(approx.reshape(4,2)[0]), font, 1, (255,0,255), 2, cv2.LINE_AA)
                                            diff = minDiff
                                            break;

                                    cv2.imshow("Corrected Perspective", warped_eq)
                                    cv2.imshow("Matching Operation", diffImg)
                                    cv2.imshow("Contours", edges)

            #Display Main Frame
            cv2.imshow("Main Frame", OriginalFrame)

            k = cv2.waitKey(1) & 0xFF
            if k == 27:
                    break

    video.release()
    cv2.destroyAllWindows()


#Run Main
if __name__ == "__main__" :
    main()